# ECMWF Data Fetch

This study considered nearshore wind and wave conditions by using ERA5 reanalysis data from the European Centre for Medium-Range Weather Forecasts (ECMWF) through the Copernicus Climate Data Store. Please run this file using Google Colab

**Manual Setup in Google Colab**

Some required files must be manually uploaded to Google Colab before running the code. All the necessary files can be found in the current directory's subfolder named `Model_Data`.


ECMWF data must be retrieved on an island-by-island basis. Before running the codes below, please go to the subfolder corresponding to the island under the **Fetch\_data** directory, and upload the corresponding `shoreline_slope.csv` and `transects_coords.csv` files to the Colab workspace.


In [ ]:
!pip install "cdsapi>=0.7.4"
!pip install netCDF4
!pip install xarray
!pip install dask

In [ ]:
import os
import zipfile
import glob
import xarray as xr
import numpy as np
import pandas as pd
import cdsapi
import datetime
from scipy.spatial import cKDTree

In [ ]:
# Transect type options: 'radial' or 'hybrid'
transect_types = 'hybrid'

To reduce the number of files that need to be uploaded, the latitude–longitude coordinate ranges corresponding to each island are recorded here.

In [ ]:
# # Madhirivaadhoo
# sitename = 'Madhirivaadhoo'
# lat_min, lat_max = 5.267531, 5.270205
# lon_min, lon_max = 73.159007, 73.163432

# # Dhakendhoo
# sitename = 'Dhakendhoo'
# lat_min, lat_max = 5.247097, 5.249889
# lon_min, lon_max = 72.890504, 72.896411

# # Funadhoo
# sitename = 'Funadhoo'
# lat_min, lat_max = 5.271686, 5.277585
# lon_min, lon_max = 73.029058, 73.037105

# sitename = 'Aidhoo'
# lat_min, lat_max = 5.185401, 5.189496,
# lon_min, lon_max = 73.160561, 73.165986

# sitename = 'Mendhoo'
# lat_min, lat_max = 5.171016, 5.179060,
# lon_min, lon_max = 72.991709, 72.999632

sitename = 'Keyodhoo'
lat_min, lat_max = 5.274799, 5.278447,
lon_min, lon_max = 72.993681, 72.998117



if not os.path.exists("ecmwf_data"):
    os.makedirs("ecmwf_data")

Please create a `.cdsapirc` file in the user’s home directory and fill in the API key obtained from your personal application on the ECMWF website.
ECMWF website: https://www.ecmwf.int/

In [ ]:
cdsapirc_content = """
url: https://cds.climate.copernicus.eu/api
key: e752e5f4-2ec0-4884-b433-ce71651d5f86
"""

with open('/root/.cdsapirc', 'w') as f:
    f.write(cdsapirc_content)

In [ ]:
df_slope = pd.read_csv("./shoreline_slope.csv")
df_slope["date"] = pd.to_datetime(df_slope["date"])

dates_df = pd.DataFrame({
    "year": df_slope["date"].dt.year.astype(str),
    "month": df_slope["date"].dt.month.astype(str).str.zfill(2),
    "day": df_slope["date"].dt.day.astype(str).str.zfill(2)
})

dates_df = dates_df.drop_duplicates()
grouped = dates_df.groupby(["year", "month"])

In [ ]:
def open_zipped_or_netcdf(file_path, extract_dir="temp_extract"):
    """
    Open a NetCDF file directly or extract and merge from a ZIP archive.

    Returns
    -------
    xarray.Dataset
        Merged dataset from NetCDF files.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")

    with open(file_path, "rb") as f:
        start_bytes = f.read(4)

    if start_bytes.startswith(b"PK\x03\x04"):
        if not os.path.exists(extract_dir):
            os.makedirs(extract_dir, exist_ok=True)

        with zipfile.ZipFile(file_path, 'r') as z:
            z.extractall(extract_dir)

        nc_files = glob.glob(os.path.join(extract_dir, "**", "*.nc"), recursive=True)
        if not nc_files:
            raise ValueError(f"No .nc file found after decompressing {file_path}")

        ds_list = []
        for nc in nc_files:
            ds_tmp = xr.open_dataset(nc)
            ds_list.append(ds_tmp)
        ds_merged = xr.merge(ds_list)
        return ds_merged

    else:
        return xr.open_dataset(file_path)


def download_gencast_global_two_times(area, day, month, year, hours, out_folder):
    """
    Download ERA5 reanalysis single-level data and save as NetCDF.

    Returns
    -------
    str or None
        Path to the saved NetCDF file, or None if an error occurs.
    """
    c = cdsapi.Client()

    surface_vars = [
        'mean_wave_period',
        '10m_v_component_of_wind',
        '10m_u_component_of_wind',
        'significant_height_of_combined_wind_waves_and_swell',
    ]

    sl_file = f"single_levels_{year}_{month}.nc"

    try:
        if not os.path.exists(sl_file):
            print("=== Downloading ===")
            c.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'format': 'netcdf',
                    'variable': surface_vars,
                    'year': year,
                    'month': month,
                    'day': day,
                    'time': hours,
                    'area': area,
                    'grid': [0.25, 0.25],
                    'expver': '1',
                },
                sl_file
            )
            print("Surface data downloaded.\n")

        ds = open_zipped_or_netcdf(sl_file, extract_dir="temp_extract_sl")

        if 'expver' in ds.dims and ds.sizes['expver'] == 1:
            ds = ds.isel(expver=0).squeeze('expver')

        if 'number' in ds.dims and ds.sizes['number'] == 1:
            ds = ds.isel(number=0).squeeze('number')

        if 'expver' in ds.coords:
            ds = ds.drop_vars('expver')
        if 'number' in ds.coords:
            ds = ds.drop_vars('number')

        if 'valid_time' in ds.dims and 'time' not in ds.dims:
            ds = ds.rename({'valid_time': 'time'})

        rename_mapping = {
            'v10': '10m_v_component_of_wind',
            'u10': '10m_u_component_of_wind',
            'swh': 'significant_height_of_combined_wind_waves_and_swell',
            'mwp': 'mean_wave_period'
        }
        ds = ds.rename(rename_mapping)

        output_file = f"{out_folder}/{year}_{month}.nc"
        ds.to_netcdf(output_file)
        print(f"\nFinal file generated: {output_file}")

        return output_file

    except Exception as e:
        print("\n=== ERROR ===")
        print(e)
        return None


In [ ]:
if __name__ == "__main__":
    target_area = [lat_max, lon_min, lat_min, lon_max]
    out_folder = "ecmwf_data"

    for (year, month), group in grouped:
        unique_days = group["day"].unique().tolist()

        # Download at Sentinel-2 / Landset default extraction times:
        hours = ['05:00']

        print(f"Downloading ECMWF for {year}-{month} days {unique_days}")

        result_file = download_gencast_global_two_times(
            target_area,
            day=unique_days,
            month=month,
            year=year,
            hours=hours,
            out_folder=out_folder
        )
        if result_file is None:
            raise ValueError(f"Dataset download failed for {year}-{month}")


In [ ]:
df_coords = pd.read_csv("./transects_coords.csv")

transect_ids = df_coords["transect_id"].tolist()
transect_lons = df_coords["longitude"].values
transect_lats = df_coords["latitude"].values

nc_files = sorted(glob.glob("./ecmwf_data/*.nc"))
records = []


In [ ]:
for nc_path in nc_files:
    print(f"Processing: {nc_path}")
    ds = xr.open_dataset(nc_path)

    # Read longitude/latitude grid
    lons = ds["longitude"].values
    lats = ds["latitude"].values

    # Build KDTree
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    grid_points = np.column_stack([lon_grid.ravel(), lat_grid.ravel()])
    tree = cKDTree(grid_points)

    # Find the nearest grid point for each transect coordinate
    _, idxs = tree.query(np.column_stack([transect_lons, transect_lats]))
    lat_idx = idxs // len(lons)
    lon_idx = idxs % len(lons)

    variables = [
        "10m_v_component_of_wind",
        "10m_u_component_of_wind",
        "significant_height_of_combined_wind_waves_and_swell",
        "mean_wave_period"
    ]

    times = ds["time"].values

    for t_idx, t in enumerate(times):
        date_str = pd.Timestamp(t).strftime("%Y-%m-%d")

        for i, transect_id in enumerate(transect_ids):
            row = {
                "date": date_str,
                "transect_id": transect_id,
            }

            for var in variables:
                try:
                    val = ds[var].isel(time=t_idx, latitude=lat_idx[i], longitude=lon_idx[i]).values.item()
                    row[var] = val
                except:
                    row[var] = np.nan

            records.append(row)

    ds.close()


In [ ]:
df_all = pd.DataFrame(records)
df_all.to_csv(f"./ecmwf_transects_timeseries_{transect_types}.csv", index=False)

print("✅ Done! Saved as ./ecmwf_transects_timeseries.csv")


Next, please place the `ecmwf_transects_timeseries.csv` file generated by running the above code back into the subfolder corresponding to the island under the **Fetch\_data** directory.
